# Loads Bosch Heatpump Raw Data and aggregates the heatpumps

## Calling relevant libraries ##

In [16]:
import json
import pandas as pd
import numpy as np 

import pandas as pd
from retry_requests import retry

## Code to automate ##

In [3]:
bosch_full_df = pd.read_csv('data/aee_backtesting_enpal.csv', sep=';')
bosch_full_df.head()
bosch_full_df['sysid'].nunique()

1994

In [4]:
bosch_full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4694193 entries, 0 to 4694192
Data columns (total 5 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   sysid    int64 
 1   ec       object
 2   val      bool  
 3   date     object
 4   op_days  int64 
dtypes: bool(1), int64(2), object(2)
memory usage: 147.7+ MB


In [5]:
print(bosch_full_df['date'].min())
print(bosch_full_df['date'].max())

2023-05-10
2024-07-30


In [6]:
# Convert date columns to datetime and period formats
bosch_full_df['date'] = pd.to_datetime(bosch_full_df['date'])
bosch_full_df.info()
bosch_full_df['month'] = bosch_full_df['date'].dt.to_period('M')
bosch_full_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4694193 entries, 0 to 4694192
Data columns (total 5 columns):
 #   Column   Dtype         
---  ------   -----         
 0   sysid    int64         
 1   ec       object        
 2   val      bool          
 3   date     datetime64[ns]
 4   op_days  int64         
dtypes: bool(1), datetime64[ns](1), int64(2), object(1)
memory usage: 147.7+ MB


,sysid,ec,val,date,op_days,month
0,101622522,Insight4WayValveCycleWeekly,False,2024-05-24,75,2024-05
1,101622522,InsightCompressorModulationWeekly,False,2024-05-24,75,2024-05
2,101622522,InsightCompressorPerformanceWeekly,False,2024-05-24,75,2024-05
3,101622522,InsightCompressorShortCycleWeekly,False,2024-05-24,75,2024-05
4,101622522,InsightCompressorStartsDaily,False,2024-05-24,75,2024-05


In [7]:
# Pivot Bosch data and clean data
bosch_wide = bosch_full_df.pivot(index=['sysid', 'op_days', 'date'], columns='ec', values='val').reset_index().sort_values(by='sysid')
bosch_wide['date'] = pd.to_datetime(bosch_wide['date'])
bosch_wide['month'] = bosch_wide['date'].dt.to_period('M')
bosch_wide
bosch_wide.info()
bosch_wide.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 239693 entries, 0 to 239692
Data columns (total 24 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   sysid                                 239693 non-null  int64         
 1   op_days                               239693 non-null  int64         
 2   date                                  239693 non-null  datetime64[ns]
 3   Insight4WayValveCycleWeekly           239184 non-null  object        
 4   InsightCompressorModulationWeekly     239184 non-null  object        
 5   InsightCompressorPerformanceWeekly    239184 non-null  object        
 6   InsightCompressorShortCycleWeekly     239184 non-null  object        
 7   InsightCompressorStartsDaily          239693 non-null  object        
 8   InsightCompressorStartsWeekly         239184 non-null  object        
 9   InsightDefrostModeWeekly              239693 non-null  object   

ec,sysid,op_days,date,Insight4WayValveCycleWeekly,InsightCompressorModulationWeekly,InsightCompressorPerformanceWeekly,InsightCompressorShortCycleWeekly,InsightCompressorStartsDaily,InsightCompressorStartsWeekly,InsightDefrostModeWeekly,...,InsightEheaterFullPowerRuntimeWeekly,InsightEheaterRuntimeDhwWeekly,InsightEheaterRuntimeWeekly,InsightEheaterStartWeekly,InsightHeatpumpCopMonthly,InsightHeatpumpCopWeekly,InsightPc0PumpCycleWeekly,InsightPc0PumpWeekly,InsightWaterPressureWeekly,month
0,101477868,237,2023-12-14,False,False,False,False,False,False,False,...,False,False,False,False,NaN,False,False,False,False,2023-12
146,101477868,237,2024-05-08,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2024-05
147,101477868,237,2024-05-09,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2024-05
148,101477868,237,2024-05-10,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2024-05
149,101477868,237,2024-05-11,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,2024-05
150,101477868,237,2024-05-12,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,2024-05
151,101477868,237,2024-05-13,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,2024-05
152,101477868,237,2024-05-14,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,2024-05
153,101477868,237,2024-05-15,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,2024-05
154,101477868,237,2024-05-16,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,2024-05


In [15]:
# Reordering the columns, reading column names from Insights_file
column_df = pd.read_excel('Insights_Naming_Mapping.xlsx')
column_df
cols_order = column_df['New name'].to_list()
print(cols_order)
cols_order = ['sysid', 'op_days', 'date', 'month',
              'InsightEheaterRuntimeDhwWeekly',
              'InsightEheaterEnergyWeekly',
              'InsightEheaterStartWeekly',
              'InsightEheaterCycleWeekly',
              'InsightHeatpumpCopWeekly',
              'InsightHeatpumpCopMonthly',
              'InsightPc0PumpCycleWeekly',
              'InsightWaterPressureWeekly',
              'InsightEheaterFullPowerRuntimeWeekly',
              'InsightEheaterEnergyMonthly',
              'InsightEheaterRuntimeWeekly',
              'InsightCompressorModulationWeekly',
              'InsightCompressorPerformanceWeekly',
              'InsightCompressorShortCycleWeekly',
              'InsightCompressorStartsDaily',
              'InsightCompressorStartsWeekly',
              'InsightDefrostModeWeekly',
              'InsightDefrostWeekly',
              'InsightPc0PumpWeekly','Insight4WayValveCycleWeekly']
bosch_wide_clean = bosch_wide.reindex(columns=cols_order)
# bosch_wide_clean.head(10)

['InsightEheaterRuntimeDhwWeekly', 'InsightEheaterEnergyWeekly', 'InsightEheaterStartWeekly', 'InsightEheaterCycleWeekly', 'InsightHeatpumpCopWeekly', 'InsightHeatpumpCopMonthly', 'InsightPc0PumpCycleWeekly', 'InsightWaterPressureWeekly', 'InsightEheaterFullPowerRuntimeWeekly', 'InsightEheaterEnergyMonthly', 'InsightEheaterRuntimeWeekly', 'InsightCompressorModulationWeekly', 'InsightCompressorPerformanceWeekly', 'InsightCompressorShortCycleWeekly', 'InsightCompressorStartsDaily', 'InsightCompressorStartsWeekly', 'InsightDefrostModeWeekly', 'InsightDefrostWeekly', 'InsightPc0PumpWeekly', 'Insight4WayValveCycleWeekly']


In [20]:
# Ordering the columns by priority
base_cols = ['heatpumpid', 'op_days', 'date', 'month']
prio_1 = ['EheaterRuntimeDhwWeekly',
              'EheaterEnergyWeekly',
              'EheaterStartWeekly',
              'EheaterCycleWeekly',
              'HeatpumpCopWeekly',
              'HeatpumpCopMonthly',
              'Pc0PumpCycleWeekly',
              'WaterPressureWeekly']
prio_2 = ['EheaterFullPowerRuntimeWeekly',
              'EheaterEnergyMonthly',
              'EheaterRuntimeWeekly',
              'CompressorModulationWeekly',
              'CompressorPerformanceWeekly',
              'CompressorShortCycleWeekly',
              'CompressorStartsDaily',
              'CompressorStartsWeekly',
              'DefrostModeWeekly',
              'DefrostWeekly',
              'Pc0PumpWeekly']
prio_3 = ['4WayValveCycleWeekly']
col_names = ['heatpumpid', 'op_days', 'date', 'month',
              'EheaterRuntimeDhwWeekly',
              'EheaterEnergyWeekly',
              'EheaterStartWeekly',
              'EheaterCycleWeekly',
              'HeatpumpCopWeekly',
              'HeatpumpCopMonthly',
              'Pc0PumpCycleWeekly',
              'WaterPressureWeekly',
              'EheaterFullPowerRuntimeWeekly',
              'EheaterEnergyMonthly',
              'EheaterRuntimeWeekly',
              'CompressorModulationWeekly',
              'CompressorPerformanceWeekly',
              'CompressorShortCycleWeekly',
              'CompressorStartsDaily',
              'CompressorStartsWeekly',
              'DefrostModeWeekly',
              'DefrostWeekly',
              'Pc0PumpWeekly',
              '4WayValveCycleWeekly']

#bosch_wide_clean.columns = col_names
bosch_wide_clean.columns = base_cols + prio_1 + prio_2 + prio_3
bosch_wide_clean.head()

,heatpumpid,op_days,date,month,EheaterRuntimeDhwWeekly,EheaterEnergyWeekly,EheaterStartWeekly,EheaterCycleWeekly,HeatpumpCopWeekly,HeatpumpCopMonthly,...,EheaterRuntimeWeekly,CompressorModulationWeekly,CompressorPerformanceWeekly,CompressorShortCycleWeekly,CompressorStartsDaily,CompressorStartsWeekly,DefrostModeWeekly,DefrostWeekly,Pc0PumpWeekly,4WayValveCycleWeekly
0,101477868,237,2023-12-14,2023-12,False,False,False,False,False,NaN,...,False,False,False,False,False,False,False,False,False,False
146,101477868,237,2024-05-08,2024-05,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
147,101477868,237,2024-05-09,2024-05,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
148,101477868,237,2024-05-10,2024-05,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
149,101477868,237,2024-05-11,2024-05,False,False,False,False,False,False,...,False,False,True,True,False,False,False,False,False,False


In [22]:
# Creating columns to sum prio 1 and prio 2 parameters
bosch_wide_clean['prio1_count'] = bosch_wide_clean[prio_1].sum(axis=1)
bosch_wide_clean['prio2_count'] = bosch_wide_clean[prio_2].sum(axis=1)
bosch_wide_clean

,heatpumpid,op_days,date,month,EheaterRuntimeDhwWeekly,EheaterEnergyWeekly,EheaterStartWeekly,EheaterCycleWeekly,HeatpumpCopWeekly,HeatpumpCopMonthly,...,CompressorPerformanceWeekly,CompressorShortCycleWeekly,CompressorStartsDaily,CompressorStartsWeekly,DefrostModeWeekly,DefrostWeekly,Pc0PumpWeekly,4WayValveCycleWeekly,prio1_count,prio2_count
0,101477868,237,2023-12-14,2023-12,False,False,False,False,False,NaN,...,False,False,False,False,False,False,False,False,0,0
146,101477868,237,2024-05-08,2024-05,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0,0
147,101477868,237,2024-05-09,2024-05,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0,0
148,101477868,237,2024-05-10,2024-05,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0,0
149,101477868,237,2024-05-11,2024-05,False,False,False,False,False,False,...,True,True,False,False,False,False,False,False,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239680,101870699,21,2024-07-18,2024-07,False,False,False,False,False,NaN,...,True,True,False,False,False,False,False,False,0,2
239679,101870699,21,2024-07-17,2024-07,False,False,False,False,False,NaN,...,True,True,False,False,False,False,False,False,0,2
239691,101870699,21,2024-07-29,2024-07,False,False,False,False,False,NaN,...,True,True,False,False,False,False,False,False,0,2
239684,101870699,21,2024-07-22,2024-07,False,False,False,False,False,NaN,...,False,False,False,False,False,False,False,False,0,0


In [14]:
# Saving cleaned Bosch data as CSV
bosch_wide_clean.to_csv('data/bosch_wide_clean.csv', index=False)